<a href="https://colab.research.google.com/github/apolitical/colab-repoclone/blob/master/colab_repoclone_testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab-repoclone-testbed
===================

A Google Colab notebook for verifying that the `colab-repoclone` module works as intended in the real world!

First you'll have to install the latest version of the library on [PyPI](https://pypi.org/project/colab-repoclone/) using `pip`.

In [1]:
!pip install colab-repoclone -qU

Importing the library should set everything up. If your Google Drive isn't already mounted, you'll have to complete an authentication flow. Otherwise it'll probably warn you that the drive is already mounted.

The `import` statement ought to have loaded everything in `colab_repoclone/__init__.py`. This includes a method, `colab_repoclone.clone_repository()` for cloning a specified GitHub repo into your Google Colab environment.

If you don't have `vars.env` in your Google Drive, any empty one will be created at this point by the `colab_env` package. Otherwise, the variables will be loaded into the environment.

In [2]:
import colab_repoclone

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive




If all has gone to plan we ought to be able to access the module's `__version__`:

In [3]:
colab_repoclone.__version__

'0.1.1'

Now, we can try to clone a repository using the `clone_repository()` method. This method takes one input and two keywords.

The input is the link to the GitHub repository you wish to clone and edit (you *must* have proper permissions if you wish to push to this repo). The first keyword, `branch`, allows the user to specify a branch to clone and defaults to "master" if none is indicated. 

The second keyword, `method`, specifies the method of authentication. This defaults to "env" which will look in the `vars.env` file loaded by `colab_env` for the following environment variables: `GITHUB_KEY`, `USER_NAME`, `USER_EMAIL` - i.e. your GitHub personal access token that gives you permission to edit the specified repo, your GitHub username and the email linked to your GitHub account. We can check the existence of these environment variables as follows:


In [4]:
import os
print(os.getenv("USER_NAME"),os.getenv("USER_EMAIL"))

# NB: I have not retrieved the GITHUB_KEY as Google Colab stores the output of
# cells and GitHub will not allow pushing of a file that contains a visible and
# valid GitHub access code in it.

jordanml7 lueck.jordan@gmail.com


If these environment variables do not exist it will throw an error indicating so. Alternatively, the `method` keyword can be set to anything other than "env" and the user will instead be prompted to enter their valid GitHub username, email and authorization token.

Let's try doing that with the `testbed` branch of this very repository. When prompted to enter your various authentication strings you can just enter garbage - the repository will still clone but you will not have `push` permissions. Alternatively, replace the repo link with your own to confirm you can `push` to it!

In [5]:
my_repo = colab_repoclone.clone_repository("https://github.com/apolitical/colab-repoclone",branch="testbed",method="input")

Enter your GitHub Username: jordanml7
Enter your GitHub Email: lueck.jordan@gmail.com
Enter your GitHub Authorization Token: ··········


You can clone as many repositories as you want by calling the above command for each! The repo directories will be placed into the Google Colab `/content` directory and the methods for each can be accessed by referencing their respective object, such as `my_repo.push()` or `my_repo.pull()`.

Let's confirm that our repo name got set correctly:

In [6]:
my_repo.repo_name

'colab-repoclone'

You can also confirm that the link to the repo on GitHub is correct by calling `my_repo.access_repo`, however this link contains our access token so I will not do so here.

Let's check that we properly pulled the `testbed` branch:

In [7]:
my_repo.branch

'testbed'

And while we're at it, let's confirm that our username and email got set correctly:

In [8]:
print(my_repo.github_user, my_repo.github_email)

jordanml7 lueck.jordan@gmail.com


Finally, although you've probably seen it in the `Files` sidebar by now (if not, click `REFRESH`), let's confirm that our repo is now in our Colab environment:

In [9]:
!ls /content/

colab-repoclone  gdrive  sample_data


There it is, `colab-repoclone` (or your own repo if you've changed it)! Wooo!

Now you can make any changes you want to the files in `colab-repoclone`, including adding and deleting files. Let's create a dummy file that we'll throw in there:

In [0]:
dummy = open("dummy.txt","w+")

for i in range(10):
     dummy.write("This is test!\n")
    
dummy.close()

And now that we've made that *very* import file, let's move it into our repository:

In [0]:
!mv /content/dummy.txt /content/colab-repoclone/.

# Obviously you may have to change the path here if you're using your own repo

Let's confirm that it's in there:

In [12]:
!ls /content/colab-repoclone/

CHANGELOG.md	 dummy.txt  MANIFEST.in  README.md
colab_repoclone  LICENSE    Pipfile	 setup.py


There it is! We can now push our changes using `colab_repoclone`'s own `push` method!

If you're using the `colab-repoclone` repository, this will not work for you, as you do not have permission to push to it. Instead, you'll likely get a message saying:

`Command: < git push > failed. Check your permissions.`

But that's not a problem for me, because I *do* have permissions! The `push` method allows for two keywords: `commit_msg`, which is pretty self explanatory (and defaults to a generic message), and `file_path`, which can be used to specify that you only want to push files in a certain directory *within* your repo. This defaults to all files, i.e. `"."`

Let's try it:

In [0]:
my_repo.push(commit_msg="Pushing to a repo from INSIDE the repo, meta...")

Well, I didn't get any error messages, but you may have!

Now, because I don't want a stupid dummy file in my lovely repository (and because this tutorial relies on that file not existing initially), lets remove `dummy.txt` and push again!

In [0]:
!rm /content/colab-repoclone/dummy.txt

In [16]:
!ls /content/colab-repoclone/

CHANGELOG.md	 LICENSE      Pipfile	 setup.py
colab_repoclone  MANIFEST.in  README.md


And voila, it's gone.

In [0]:
my_repo.push(commit_msg="Annnddddd now its gone.")

Well that was satisfying. Currently, the only other Git functionality this package offers is `pull`, which allows you to pull changes to your repo if any were made elsewhere while you were running your notebook. Since all runtimes reset upon each load a notebook, you will need to "reclone" your repo each time you open and run your Colab notebook, so pulling will rarely be necessary.

And that's all! Have fun and feel free to email me with questions or if you'd like to see any new functionality in a future release!